In [1]:
! pip install pandas --break-system-packages && pip install sqlalchemy --break-system-packages && pip install psycopg2-binary --break-system-packages

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

In [3]:
load_dotenv('../database/.env')

usuario_database = os.getenv('DB_USER')
senha_database = os.getenv('DB_PASSWORD')
host_database = os.getenv('DB_HOST')
port_database = os.getenv('DB_PORT')
name_database = os.getenv('DB_NAME')

DATABASE_URL = f'postgresql://{usuario_database}:{senha_database}@{host_database}:{port_database}/{name_database}'
#print(DATABASE_URL)
engine = create_engine(DATABASE_URL)


In [4]:
raw_data_dict = {
    '2021_receitas_raw_data' : 'data/recife-dados-receitas-2021.csv',
    '2022_receitas_raw_data' : 'data/recife-dados-receitas-2022.csv',
    '2023_receitas_raw_data' : 'data/recife-dados-receitas-2023.csv'
}

In [5]:
try:    
    display(pd.read_sql('SELECT 1', engine))
except:
    print('A conexão com o banco falhou.')

,?column?
0,1


In [8]:
for nome_tabela, caminho in raw_data_dict.items():
    try:
        df_raw = pd.read_csv(caminho, on_bad_lines='warn', sep=';')

        df_raw.to_sql(
            nome_tabela,
            con=engine,
            if_exists='replace',
            index=False
        )

    except Exception as e:
        print(f'erro {e}')